the classes will eventually be split up but they are put together for now to get their interactions correct <br>
sources: <br>
events: https://www.quantstart.com/articles/Event-Driven-Backtesting-with-Python-Part-II/ <br>
portfolio: https://www.quantstart.com/articles/Event-Driven-Backtesting-with-Python-Part-V/ <br>
execution handler: https://www.quantstart.com/articles/Event-Driven-Backtesting-with-Python-Part-VI/

In [1]:
import numpy as np
import pandas as pd
import os

from abc import ABC, abstractmethod
from queue import PriorityQueue
from dataclasses import dataclass
from functools import total_ordering

In [2]:
quotes_filename = os.path.join("play_data", "XBTUSD_quotes_191214_0434.csv")
trades_filename = os.path.join("play_data", "XBTUSD_trades_191214_0434.csv")

In [3]:
quotes_full = pd.read_csv(
    quotes_filename,
    index_col='recorded',
    parse_dates=['timestamp', 'recorded']
)

trades_full = pd.read_csv(
    trades_filename,
    index_col='received',
    parse_dates=['timestamp', 'received'],
    dtype={"side":"category"}
)

all_data = {
    "quotes": quotes_full,
    "trades": trades_full
}

display( quotes_full.head() )
display( trades_full.head() )

,timestamp,bidSize,bidPrice,askPrice,askSize
recorded,,,,,
2019-12-14 04:34:40.731941+00:00,2019-12-14 04:34:40.024000+00:00,3840427,7251.5,7252.0,701477
2019-12-14 04:34:41.211565+00:00,2019-12-14 04:34:40.410000+00:00,3840391,7251.5,7252.0,701477
2019-12-14 04:34:42.210955+00:00,2019-12-14 04:34:41.165000+00:00,3840391,7251.5,7252.0,731477
2019-12-14 04:34:42.210955+00:00,2019-12-14 04:34:41.183000+00:00,3840187,7251.5,7252.0,731277
2019-12-14 04:34:42.210955+00:00,2019-12-14 04:34:41.431000+00:00,3840187,7251.5,7252.0,715277


,timestamp,side,size,price
received,,,,
2019-12-14 04:34:40.742081+00:00,2019-12-14 04:34:38.713000+00:00,Sell,2500,7251.5
2019-12-14 04:34:42.499730+00:00,2019-12-14 04:34:42.378000+00:00,Buy,12000,7252.0
2019-12-14 04:34:48.168746+00:00,2019-12-14 04:34:48.044000+00:00,Buy,100000,7252.0
2019-12-14 04:34:48.332053+00:00,2019-12-14 04:34:48.213000+00:00,Buy,116,7252.0
2019-12-14 04:34:50.665331+00:00,2019-12-14 04:34:50.504000+00:00,Buy,2500,7252.0


In [4]:
@total_ordering
class Event(ABC):
    def __post_init__(self):
        assert type(self.event_time) == pd.Timestamp,\
        "pd.Timestamp event_time required"
    def __eq__(self, other):
        return self.event_time == other.event_time
    def __lt__(self, other):
        return self.event_time < other.event_time
    
        
class EventQueue(PriorityQueue):
    def __init__(self, start_time, *args, **kwargs):
        self.current_time = start_time
        return super().__init__(*args, **kwargs)
        
    def get(self, *args, **kwargs):  # may be bad for performance
        event = super().get(*args, **kwargs)
        self.current_time = event.event_time
        return event
    
    
@dataclass
class FillEvent(Event):
    __slots__ = "event_time", "symbol", "exchange", "quantity", "cost"
    event_time : pd.Timestamp
    symbol: str
    exchange: str
    quantity: int  # change in securities held
    cost: float  # change in cash held
        
    @property
    def commission(self):
        # will eventually contain exchange/broker-specific logic
        return self.cost * 0.005
    
@dataclass
class Signal:
    __slots__ = "symbol", "exchange", "signal_type", "strength"
    symbol: str
    exchange: str
    signal_type: str  # BUY/SELL/EXIT
    strength: float
    
@dataclass
class Order:
    __slots__ = "symbol", "exchange", "order_type", "direction", "quantity", "price"
    symbol: str
    exchange: str
    order_type: str  # MKT/LMT
    direction: str  # BUY/SELL
    quantity: int
    price: float  # only relevent to LMT

In [5]:
class Portfolio(ABC):
    @abstractmethod
    def update_fill(self):
        pass
    
    @abstractmethod
    def send_order_from_signal(self):
        pass
    
class BacktestPortfolio(Portfolio):
    def __init__(self, queue, symbol_list, all_data, initial_capital=10000.0):
        self.queue = queue  # event queue
        self.symbol_list = symbol_list 
        self.initial_capital = initial_capital
        
        start_time = self.queue.current_time
        self.all_positions_list = [pd.Series(0, index=symbol_list, name=start_time)]  # list of series
        self.all_holdings_list = [self.construct_current_holdings()]  # list of series
        
        self.current_positions = pd.Series(0, index=symbol_list, name=start_time)
        self.current_holdings = self.construct_current_holdings()  # series

    @property
    def all_positions(self):
        return pd.DataFrame(self.all_positions_list)
    
    @property
    def all_holdings(self):
        return pd.DataFrame(self.all_holdings_list)
    
    def construct_current_holdings(self):  # market values of positions
        holdings = {symbol: 0.0 for symbol in symbol_list}
        holdings["capital"] = self.initial_capital
        holdings["commission"] = 0.0
        holdings["total"] = self.initial_capital
        return pd.Series(holdings, name=self.queue.current_time)
    
    def update_fill(self, fill):
        # maybe can split position and holding updates
        self.current_positions.name = self.queue.current_time
        self.current_positions[fill.symbol] += fill.quantity
        self.all_positions_list.append(self.current_positions.copy())
        
        self.current_holdings.name = self.queue.current_time
        self.current_holdings["capital"] -= fill.cost
        # assuming constant prices between txns
        self.current_holdings[fill.symbol] = self.current_positions[fill.symbol] * fill.cost
        self.current_holdings["commission"] += fill.commission
        self.all_holdings_list.append(self.current_holdings.copy())
        
    def risk_check(self):
        return True
    
    def calculate_order_price(self, order_type):
        # relevant to limit orders
        if order_type == "MKT":
            return 0.0
        return 0.0
    
    def send_order_from_signal(self, sig):
        order_type = "MKT"
        if sig.signal_type == "EXIT":
            if self.current_positions[signal.symbol] < 0:
                direction = "BUY"
            else:
                direction = "SELL"
        else:
            direction = sig.signal_type
            
        quantity = int(10 * sig.strength)
        price = self.calculate_order_price(order_type)
        
        if self.risk_check():
            self.generate_fill_from_order(
                Order(sig.symbol, sig.exchange, order_type, direction, quantity, price)
            )
    
    def calculate_fill_cost(self, order):
        fill_price = 7000  # will eventually use future data
        return order.quantity * fill_price

    def calculate_fill_quantity(self, order):
        return order.quantity  # assumes 100% fill
    
    def calculate_latency(self, order):
        return pd.Timedelta("3s")
    
    def generate_fill_from_order(self, order):
        event_time = self.queue.current_time + self.calculate_latency(order)
        quantity = self.calculate_fill_quantity(order)
        cost = self.calculate_fill_cost(order)
        
        fill = FillEvent(event_time, order.symbol, order.exchange, quantity, cost)
        self.queue.put(fill)

In [6]:
import time
symbol_list = "XBTUSD", "ETHUSD"
time0 = pd.Timestamp.utcnow() - pd.Timedelta("15s")

eq = EventQueue(time0)
p = BacktestPortfolio(eq, symbol_list, all_data, 1000000)
display(p.all_positions)
display(p.all_holdings)

s0 = Signal("XBTUSD", "BitMex", "BUY", 5.0)
s1 = Signal("XBTUSD", "BitMex", "BUY", 15.0)

p.send_order_from_signal(s0)
time.sleep(1)
p.send_order_from_signal(s1)

while not eq.empty():
    event = eq.get()
    if type(event) == FillEvent:
        p.update_fill(event)
    display(p.all_positions)
    display(p.all_holdings)

,XBTUSD,ETHUSD
2020-01-12 08:56:26.664216+00:00,0,0


,XBTUSD,ETHUSD,capital,commission,total
2020-01-12 08:56:26.664216+00:00,0.0,0.0,1000000.0,0.0,1000000.0


,XBTUSD,ETHUSD
2020-01-12 08:56:26.664216+00:00,0,0
2020-01-12 08:56:29.664216+00:00,50,0


,XBTUSD,ETHUSD,capital,commission,total
2020-01-12 08:56:26.664216+00:00,0.0,0.0,1000000.0,0.0,1000000.0
2020-01-12 08:56:29.664216+00:00,17500000.0,0.0,650000.0,1750.0,1000000.0


,XBTUSD,ETHUSD
2020-01-12 08:56:26.664216+00:00,0,0
2020-01-12 08:56:29.664216+00:00,50,0
2020-01-12 08:56:29.664216+00:00,200,0


,XBTUSD,ETHUSD,capital,commission,total
2020-01-12 08:56:26.664216+00:00,0.0,0.0,1000000.0,0.0,1000000.0
2020-01-12 08:56:29.664216+00:00,17500000.0,0.0,650000.0,1750.0,1000000.0
2020-01-12 08:56:29.664216+00:00,210000000.0,0.0,-400000.0,7000.0,1000000.0
